#### mobileNet

#### importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

: 

#### 1. CONFIGURATION & HYPERPARAMETERS ---

#### load model

In [ ]:
base_model = MobileNetV2(weights='imagenet',
                          include_top=False, 
                          input_shape=(224, 224, 3))
base_model.trainable = False   # Freeze layers


Found 46649 images belonging to 16 classes.
Found 11683 images belonging to 16 classes.


#### 3. add custom layer

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


##### model combiling 

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


#### 5.prepare data set

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



--- PHASE 1: Training the New Classification Head ---
Epoch 1/5


c:\Users\user\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


##### 6. train the model 

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


### Fine tune base model

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3
)


#### save your model

In [ ]:
model.save("mobilenet_model10.h5")
